In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import datetime
import itertools
import h5py

In [2]:
def reformat(log, pred):
    #concatenate the log (t15 - t0) and pred images (t1 - t15)
    #shape [example, image, height, width, channels]
    output = np.concatenate((log, pred), axis=1) #concat along the image axis
    H = output.shape[2]
    W = output.shape[3]
    C = output.shape[4]
    output = np.reshape(output, [-1, H, W, C])
    return output
    

In [3]:
# define the data location and load data
# cwd = os.getcwd()

data_folder = '/scratch/groups/abrandt/GAN_project/data/benchmark_models_initial_testing'
image_log_train = np.load(os.path.join(data_folder,'images_log_train_model_test.npy'))
image_pred_train = np.load(os.path.join(data_folder,'images_pred_train_model_test.npy'))
print('Train data: ', image_log_train.shape)

num_examples_train = image_log_train.shape[0]
train_data = reformat(image_log_train, image_pred_train)
del image_log_train, image_pred_train
print('Train data refomatted: ', train_data.shape)

image_log_val = np.load(os.path.join(data_folder,'images_log_val_model_test.npy'))
image_pred_val = np.load(os.path.join(data_folder,'images_pred_val_model_test.npy'))
print('Val data: ', image_log_val.shape)

num_examples_val = image_log_val.shape[0]
val_data = reformat(image_log_val, image_pred_val)
print('Val data refomatted: ', val_data.shape)

del image_log_val, image_pred_val

Train data:  (18, 16, 64, 64, 3)
Train data refomatted:  (558, 64, 64, 3)
Val data:  (2, 16, 64, 64, 3)
Val data refomatted:  (62, 64, 64, 3)


In [5]:
#save to HDF5 file foramat
num_images = 16+15

hf =  h5py.File("GPT_benchmark.hdf5", "w") #create file
hf.create_dataset('train_data', data=train_data)

print('Num_examples_train: ', num_examples_train)
train_idx = np.arange(0, num_examples_train * num_images, num_images) #start index for each example
print('Train idx: ', train_idx)
hf.create_dataset('train_idx', data=train_idx)

hf.create_dataset('test_data', data=val_data) #NOTE: this is actually the validation data (only do this for the benchamark since no test)

print('Num_examples_val: ', num_examples_val)
val_idx = np.arange(0, num_examples_val * num_images, num_images) #start index for each example
print('Val idx: ', val_idx)
hf.create_dataset('test_idx', data=val_idx)

hf.close()

Num_examples_train:  18
Train idx:  [  0  31  62  93 124 155 186 217 248 279 310 341 372 403 434 465 496 527]
Num_examples_val:  2
Val idx:  [ 0 31]
